# Writing tests

> How to write tests with `nbmodular`

In [ ]:
%load_ext nbmodular.core.cell2func

## Implement functions

In [ ]:
%%function data
a=1
b=2

In [ ]:
%%function add_function
c = a+b

## Writing tests with dependencies

It is best to define data variables in separate data functions, so that they can be retrieved and reused from different test functions:

In [ ]:
%%function first_data --test --data
a = 1

In [ ]:
%%function second_data --test --data
b = 2

Then we can define our test function. This function uses the variables defined in our previous data functions:

Before trying to test a previous function we need to ensure that its output is the required one:

In [ ]:
%print add_function

def add_function(a, b):
    c = a+b



As we can see `add_function` still doesn't return anything because there were no other functions depending on it. The way to create a function depending on it is to use any of its created variables in another cell function, like so:

```python
%%function uses_c
print (c)
```
Since we won't be needing such function for the time being, we can just manually add this dependency with the magic `add_to_signature`:

In [ ]:
%add_to_signature add_function --output c

Now `add_function` has the required output:

In [ ]:
%print add_function

def add_function(a, b):
    c = a+b
    return c



And we can call it in our test function:

In [ ]:
%%function add_function --test
c = add_function (a, b)
assert c==3

Let's look at the resulting implementation of our test function:

In [ ]:
%print test_add_function --test

def test_add_function():
    a = test_first_data()
    b = test_second_data()
    c = add_function (a, b)
    assert c==3



If we try to define the same variable in another data function, we will get an error:

In [ ]:
%%function third_data --test --data
b = 3

ValueError: detected common variables with other test data functions {'b'}:

In [ ]:
%print_pipeline --test

In [ ]:
test_test_tutorial_pipeline 

In [ ]:
test_test_tutorial_pipeline ()

In [ ]:
test_tutorial_pipeline ()

In [ ]:
tutorial_pipeline 

In [ ]:
c = %cell_processor

In [ ]:
c.pipeline